<a href="https://colab.research.google.com/github/qriovider/queueing_theory/blob/main/JSQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpy

In [7]:
import simpy
import numpy as np

In [64]:
LAMBDA = 4
MU = 5

def generate_interarrival():
  return np.random.exponential(1./LAMBDA)

def generate_service():
  return np.random.exponential(1./MU)

def arriving(env, counters):
  i = 0
  while True:
    i = i + 1
    yield env.timeout(generate_interarrival())
    env.process(customer(env, i, counters))

def customer(env, i, counters):
  arrive = env.now
  Qlength = [NoInSystem(counters[k]) for k in range(len(counters))]
  print(env.now, 'customer{} arrives'.format(i), Qlength)
  for k in range(len(Qlength)):
      if Qlength[k] == 0 or Qlength[k] == min(Qlength):
        choice = k  # the customer chooses a server which has the minimum number of queues 
        break
  with counters[choice].request() as req:
    # the customer wait for the counter
    yield req
    wait = env.now - arrive
    # the customer got to the counter
    print(env.now, 'customer{0} waited for {1}s'.format(i, wait))
    print(env.now, 'customer{} is being served'.format(i))
    yield env.timeout(generate_service())
    print(env.now, 'customer{} departs'.format(i))

def NoInSystem(R):
    """Total number of customers in the resource R"""
    return max([0, len(R.put_queue) + len(R.users)])

In [65]:
np.random.seed(0)
env = simpy.Environment()
counters = [simpy.Resource(env), simpy.Resource(env)]
env.process(arriving(env, counters))
env.run(until=50)

0.1989686270407775 customer1 arrives [0, 0]
0.1989686270407775 customer1 waited for 0.0s
0.1989686270407775 customer1 is being served
0.38361325620159126 customer1 departs
0.5129513177822369 customer2 arrives [0, 0]
0.5129513177822369 customer2 waited for 0.0s
0.5129513177822369 customer2 is being served
0.6231610160013368 customer2 departs
0.7097516058615546 customer3 arrives [0, 0]
0.7097516058615546 customer3 waited for 0.0s
0.7097516058615546 customer3 is being served
0.8248554456952826 customer3 departs
0.969291429597457 customer4 arrives [0, 0]
0.969291429597457 customer4 waited for 0.0s
0.969291429597457 customer4 is being served
1.5251725328556212 customer5 arrives [1, 0]
1.5251725328556212 customer5 waited for 0.0s
1.5251725328556212 customer5 is being served
1.6322738660082199 customer4 departs
1.6460730580811844 customer6 arrives [0, 1]
1.6460730580811844 customer6 waited for 0.0s
1.6460730580811844 customer6 is being served
1.813959627428949 customer6 departs
1.834241585465